The purpose of this notebook is to lay out the intended behaviors for the redesign of `lamatrix`.

Written by Rae Holcomb, 7/26/2024

# The Generator Class




# Offset Generators

An _OffsetGenerator_ represents a column of ones in the design matrix. This represents a constant offset to the model. When building a model, you should have no more than one _OffsetGenerator_. (I think we should consider building in a `_validate()` function which raises a warning if you try to add an _OffsetGenerator_ to a model that already has one.) 

By convention, when combining generators we put the _OffsetGenerator_ first. Thus a model representing a sinusoid with a constant offset
$$ M = 1 + \sin(\theta)$$
would be represented as
$$ M = SIG(\text{OffsetGenerator}, \text{SineGenerator})$$
where SIG represents a stacked independent generator.

One unique behavior is that an _OffsetGenerator_ does not require any input args to create. As a result, you can not create the design matrix for an _OffsetGenerator_ until you combine it with other generators that do have input args. After that, the _OffsetGenerator_ will infer the data shape from those args.

# Combinding Generators Independently

This should work more or less the same way as the original verion of `lamatrix`. 

Note: Stacking a SIG with another generator adds them all on the same level. It does _not_ create an SIG nested inside another SIG. The intended behavior is:

$$SIG(A, B) + C → SIG(A, B, C) $$

**NOT**

$$SIG(A, B) + C → SIG(SIG(A, B), C) $$

As a result of this (and the changes to dependent combinations), this means you should never have generators nested deeper than one level.


# Combinding Generators Dependently

Multiplying two generators performs the following operation:
$$ a * b = a + b + \text{CrosstermGenerator}(a,b) $$
which is the same as
$$ a * b = SIG(a, b, CG(a,b)) $$

Note:
* A _CrosstermGenerator_ is a special class which contains the crossterms of two or more generators. 
* The `width` of a CG will be the product of the widths of its input generators.
* We may want to build in a warning that either prevents an _OffsetGenerator_ from being passed as an input to a _CrosstermGenerator_.

## Examples
Here are some examples of the expected behaviors for more complex combinations.

Example 1:
$$ a * b * c = (a + b + CG(a,b)) * c $$
$$ = a + b + c + CG(a,b) + CG(a,c) + CG(b,c) + CG(a,b,c) $$

Example 2:
$$ (a + b) * c = a + b + c + CG(a,c) + CG(b,c) $$

The way I think about implementing this is that you copy the full first input to the _CrosstermGenerator_, copy the full second input, and then add a _CrosstermGenerator_ for every combination of generators in the first input and the second input.



## Edge Cases

**Edge Case 1: Only One Input**

This should probably just through an error, but it is also a possibility that a _CrosstermGenerator_ with only one generator inputted defaults to just returning the generator. Thus $CG(a)$ would return $a$.


**Edge Case 2: OffsetGenerators**
We will need to think about the behavior of _OffestGenerator_s when passed to a _CrosstermGenerator_. I see two possibilities:
1. Completely disallow it. Raise a warning if an a _OffsetGenerator_ is passed as an input.
2. _OffsetGenerator_ inputs are ignored. So 
$$CG(\text{OffestGenerator}, b, c) → CG(b, c) $$

I'm leaning towards the first behavior, since the second could lead to redundancies like this

$$a * (b+1) → a + b + 1 + CG(a,b) + CG(a,1) → 1 + b + b$$

# Setting Priors

In complex models, we should encourage the user to index into the base generator they want in order to set priors. For example, for a model like this
$$ p_{1} = \text{PolynomialGenerator}(x, polyorder=2) $$
$$ p_{2} = \text{PolynomialGenerator}(y, polyorder=2) $$
$$ M = p_{1} + p_{2} = SIG(p_{1}, p_{2}) $$
the correct way to set a prior on the first order term of $x$ in $M$ would be

`M[0].set_prior(0, (mean,std))`

This means that **order matters** inside of a SIG, and should be preserved in an intuitive way.

To make things easier for the user, I was also planning to build a helper function called `Generator.get_term(index)` which would return the term of the equation corresponding the element at that index within a generator. I think it would be helpful when checking to make sure you're setting the prior on the right term!
